In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import time

#Random Forest
from sklearn.ensemble import RandomForestClassifier
# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV, StratifiedKFold


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin, clone

from scipy.stats import ttest_rel
from scipy.stats import randint, uniform

#hiperparamentros search
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from scipy.stats import randint, uniform,loguniform


# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_diab import *


cv_strat = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))



#Processo finalizado em: 20:03:52


## 1. Load Data & Pipeline

In [2]:
BASE = Path.cwd().parent

PP2 = joblib.load(BASE/'src'/'preprocess_diabetes_v1.2.joblib')['preprocessador']

DATA_DIR = BASE/"data"/"raw"
X_train = pd.read_csv(DATA_DIR/"X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR/"X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR/"y_train_raw.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR/"y_test_raw.csv").values.ravel()

## 2.Baseline

In [3]:
# Baseline
#model_base = RandomForestClassifier(random_state=42, n_jobs=4)
model_base = HistGradientBoostingClassifier(random_state=42,early_stopping=True)
pipe_base  = pipe_models(model_base, PP2) # Pipiline base ( PP2 é o preprocessador)

cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

pipe_base.fit(X_train, y_train)
y_probs0 = pipe_base.predict_proba(X_test)[:, 1]
# 2. Testar vários thresholds
thresholds = np.linspace(0.3, 0.7, 41)
best_threshold0 = 0.5
max_acc = 0

for t in thresholds:
    acc = accuracy_score(y_test, y_probs0 > t)
    if acc > max_acc:
        max_acc = acc
        best_threshold0 = t
print(f"{'='*70}")
print(f"🎯 Melhor Threshold: {best_threshold0:.3f}")
print(f"📈 Melhor Acurácia de Teste: {max_acc:.4f}")
print(f"{'='*70}")

baseline_scores = cross_val_score(pipe_base, X_train, y_train, cv=10)
print(f"{'='*70}")
print(f"Baseline: {baseline_scores.mean():.4f} ± {baseline_scores.std():.4f}")
print(f"Average CV Accuracy: {np.mean(baseline_scores)*100:.2f}%") 

mtd_scoring='roc_auc'
#accuracy

🎯 Melhor Threshold: 0.500
📈 Melhor Acurácia de Teste: 0.6801
Baseline: 0.6797 ± 0.0014
Average CV Accuracy: 67.97%


## 3.Buscas por hiperparamentros


In [4]:
#  (Exploratória)
param_dist_1 = {
    'model__max_iter': randint(100, 1500),         # De poucas árvores a muitas
    'model__learning_rate': loguniform(0.003, 0.3),    #lento a agressivo
    'model__max_leaf_nodes': randint(15, 255),     # De árvores simples a muito complexas (HGBC lida bem com isso)
    'model__max_depth': randint(2, 15),    # Permitir árvores mais profundas se necessário
    'model__l2_regularization': loguniform(1e-4, 5.0),    # Regularização variada
    'model__min_samples_leaf': randint(20, 200)     # Amostras por folha para evitar overfitting em árvores grandes

}
numero_itera=50
cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

search_1 = RandomizedSearchCV(
    pipe_base, param_dist_1,
    n_iter=numero_itera, cv=cv_s,
    scoring=mtd_scoring,
    random_state=42, verbose=1
)

start = time.time()
search_1.fit(X_train, y_train)
end = time.time()

best_1 = search_1.best_estimator_


# 2. Testar vários thresholds
y_probs1 = best_1.predict_proba(X_test)[:, 1]
thresholds = np.linspace(0.3, 0.7, 41)
best_threshold1 = 0.5
max_acc = 0

for t in thresholds:
    acc = accuracy_score(y_test, y_probs1 > t)
    if acc > max_acc:
        max_acc = acc
        best_threshold1 = t
print(f"{'='*70}")
print(f"🎯 Melhor Threshold: {best_threshold1:.3f}")
print(f"📈 Melhor Acurácia de Teste: {max_acc:.4f}")
print(f"{'='*70}")

#
##scores1 = cross_val_score(best_1, X_train, y_train, cv=10)
print(f"{'='*70}")
#print(f"Optimized: {scores1.mean():.4f} ± {scores1.std():.4f}")
#print(f"Average CV Accuracy: {np.mean(scores1)*100:.2f}%")
print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/numero_itera:.2f} segundos")
print("📌 Melhores Parâmetros:")
print(search_1.best_params_)
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))

Fitting 3 folds for each of 50 candidates, totalling 150 fits
🎯 Melhor Threshold: 0.510
📈 Melhor Acurácia de Teste: 0.6824
Tempo total: 3245.45 segundos
Tempo por iteração: 64.91 segundos
📌 Melhores Parâmetros:
{'model__l2_regularization': 0.00014050274230636859, 'model__learning_rate': 0.056226632956667064, 'model__max_depth': 5, 'model__max_iter': 1120, 'model__max_leaf_nodes': 110, 'model__min_samples_leaf': 151}

#Processo finalizado em: 20:59:19


In [5]:

start = time.time()
scores1 = cross_val_score(best_1, X_train, y_train, cv=10)
end = time.time()
print(f"{'='*70}")
print(f"Optimized: {scores1.mean():.4f} ± {scores1.std():.4f}")
print(f"Average CV Accuracy: {np.mean(scores1)*100:.2f}%")
print(f"Tempo total: {end-start:.2f} segundos")



Optimized: 0.6822 ± 0.0014
Average CV Accuracy: 68.22%
Tempo total: 251.31 segundos


In [6]:
def generate_refined_grid(best_params):
    """
    Gera um grid refinado automaticamente baseado nos melhores parâmetros,
    organizando a lógica antes de retornar o dicionário.
    """
    # 1. Cálculos de limites para Inteiros (Max Iter, Nodes, Samples)
    iter_min, iter_max = int(best_params['model__max_iter'] * 0.8), int(best_params['model__max_iter'] * 1.2)
    nodes_min, nodes_max = max(20, int(best_params['model__max_leaf_nodes'] * 0.7)), int(best_params['model__max_leaf_nodes'] * 1.3)
    leaf_min, leaf_max = int(best_params['model__min_samples_leaf'] * 0.8), int(best_params['model__min_samples_leaf'] * 1.3)

    # 2. Lógica para Max Depth (evita erro se for None)
    current_depth = best_params['model__max_depth']
    if current_depth is not None:
        depth_list = [current_depth - 1, current_depth, current_depth + 1]
    else:
        depth_list = [None, 10, 20]

    # 3. Definição do Grid
    refined_grid = {
        'model__max_iter': randint(iter_min, iter_max + 1),
        
        'model__learning_rate': uniform(best_params['model__learning_rate'] * 0.7, 
                                        best_params['model__learning_rate'] * 0.6),
        
        'model__max_leaf_nodes': randint(nodes_min, nodes_max + 1),
        
        'model__max_depth': depth_list,
        
        'model__min_samples_leaf': randint(leaf_min, leaf_max + 1),
        
        'model__l2_regularization': uniform(max(0, best_params['model__l2_regularization'] * 0.5), 1.0)
    }


    lr_start = best_params['model__learning_rate'] * 0.7
    lr_end = lr_start + (best_params['model__learning_rate'] * 0.6)

    l2_start = max(0, best_params['model__l2_regularization'] * 0.6)
    l2_end = lr_start + 1

    # 4. Verificação Visual (Print dos Intervalos)
    print(f"{'='*30} NOVO GRID DE REFINO {'='*30}")
    print(f"🔹 Max Iter: {iter_min} até {iter_max}")

    print(f"🔹 Learning Rate: {lr_start:.4f} até {lr_end:.4f}")
    print(f"🔹 Max Leaf Nodes: {nodes_min} até {nodes_max}")
    print(f"🔹 Max Depth: {depth_list}")
    print(f"🔹 Min Samples Leaf: {leaf_min} até {leaf_max}")
    print(f"🔹 l2 reg: {l2_start:.4f} até {l2_end:.4f}")

    print(f"{'='*81}\n")

    return refined_grid

# # Aplicação automática
param_dist_2 = generate_refined_grid(search_1.best_params_)

# #  refino
# param_dist_2 = {
#     # Busca granular entre os valores que funcionaram bem
#     'model__max_iter': randint(300, 600),
#     'model__learning_rate': uniform(0.02, 0.12), # Cobre de 0.02 a 0.14
#     'model__max_leaf_nodes': randint(40, 121),
#     'model__max_depth': randint(4,8),
#     'model__min_samples_leaf': randint(80, 121),
#     'model__l2_regularization': [0.05, 0.1, 0.2]
#}

# Reduzi o CV para 3 para você poder testar mais candidatos (n_iter) 
numero_itera=30
search_2 = RandomizedSearchCV(
    pipe_base, param_dist_2,
    n_iter=numero_itera, cv=cv_s, 
    scoring=mtd_scoring,
    random_state=42, verbose=1,n_jobs=4
)

start = time.time()
search_2.fit(X_train, y_train)
end = time.time()

best_2 = search_2.best_estimator_


# # 2. Testar vários thresholds
y_probs2 = best_2.predict_proba(X_test)[:, 1]
thresholds = np.linspace(0.3, 0.7, 41)
best_threshold2 = 0.5
max_acc2 = 0

for t in thresholds:
    acc = accuracy_score(y_test, y_probs2 > t)
    if acc > max_acc2:
        max_acc2 = acc
        best_threshold2 = t
print(f"{'='*70}")
print(f"🎯 Melhor Threshold: {best_threshold2:.3f}")
print(f"📈 Melhor Acurácia de Teste: {max_acc2:.4f}")
print(f"{'='*70}")

#
##scores1 = cross_val_score(best_1, X_train, y_train, cv=10)
print(f"{'='*70}")
#print(f"Optimized: {scores1.mean():.4f} ± {scores1.std():.4f}")
#print(f"Average CV Accuracy: {np.mean(scores1)*100:.2f}%")
print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/numero_itera:.2f} segundos")
print("📌 Melhores Parâmetros:")
print(search_2.best_params_)
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))

============================== NOVO GRID DE REFINO ==============================
🔹 Max Iter: 896 até 1344
🔹 Learning Rate: 0.0394 até 0.0731
🔹 Max Leaf Nodes: 77 até 143
🔹 Max Depth: [4, 5, 6]
🔹 Min Samples Leaf: 120 até 196
🔹 l2 reg: 0.0001 até 1.0394

Fitting 3 folds for each of 30 candidates, totalling 90 fits
🎯 Melhor Threshold: 0.510
📈 Melhor Acurácia de Teste: 0.6827
Tempo total: 1361.65 segundos
Tempo por iteração: 45.39 segundos
📌 Melhores Parâmetros:
{'model__l2_regularization': 0.005592368494755583, 'model__learning_rate': 0.06686903332649657, 'model__max_depth': 4, 'model__max_iter': 1287, 'model__max_leaf_nodes': 111, 'model__min_samples_leaf': 154}

#Processo finalizado em: 21:26:16


In [7]:
start = time.time()
scores2 = cross_val_score(best_2, X_train, y_train, cv=10)
end = time.time()
print(f"{'='*70}")
print(f"Optimized: {scores2.mean():.4f} ± {scores2.std():.4f}")
print(f"Average CV Accuracy: {np.mean(scores2)*100:.2f}%")
print(f"Tempo total: {end-start:.2f} segundos")


Optimized: 0.6819 ± 0.0015
Average CV Accuracy: 68.19%
Tempo total: 219.19 segundos
